Usando o módulo DEAP sem a função eaSimple
==========================================



## Introdução



A função `eaSimple` é excelente para resolver algoritmos genéticos de maneira rápida e fácil, porém ela não é flexível. Não podemos, por exemplo, usar um outro critério de parada que não o de número de gerações.

Podemos usar o `DEAP` sem a função `eaSimple`! Já sabemos os passos de como funciona um algoritmo genético, basta construir isso usando as ferramentas do `DEAP`.



## Importações



In [1]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools

from funcoes import gene_cb

## Problema das caixas binárias usando `DEAP` sem a função `eaSimple`



Antes de iniciar o problema, vamos importar as funções necessárias e definir as constantes.



In [19]:
# relacionadas ao problema a ser resolvido
NUM_CAIXAS = 100

# relacionadas à busca
TAMANHO_POP = 90
NUM_GERACOES = 10000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
CHANCE_MUTACAO_DE_CADA_GENE = 1 / NUM_CAIXAS
NUM_COMBATENTES_NO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Precisamos definir a função objetivo.



In [20]:
def funcao_objetivo_cb(individuo):
    """Computa a função objetivo no problema das caixas binárias.

    Args:
      individiuo: lista contendo os genes das caixas binárias

    Return:
      Uma tupla com o valor representando a soma dos genes do individuo.
    """
    return (sum(individuo), )

Vamos faver as definições que o `DEAP` necessita de forma similar ao que fizemos no experimento anterior. Veja que até aqui não tem nada de diferente do que já vimos!



In [21]:
creator.create("Fitness_max", base.Fitness, weights=(1.0,))
creator.create("Individuo", list, fitness=creator.Fitness_max)

toolbox = base.Toolbox()

toolbox.register(
    "individuo", tools.initRepeat, creator.Individuo, gene_cb, NUM_CAIXAS
)

toolbox.register(
    "populacao", tools.initRepeat, list, toolbox.individuo, TAMANHO_POP
)

toolbox.register("evaluate", funcao_objetivo_cb)

toolbox.register(
    "select", tools.selTournament, tournsize=NUM_COMBATENTES_NO_TORNEIO
)

toolbox.register("mate", tools.cxOnePoint)

toolbox.register("mutate", tools.mutFlipBit, indpb=CHANCE_MUTACAO_DE_CADA_GENE)

hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("avg", np.mean)
estatisticas.register("std", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

log = tools.Logbook()

c:\Users\guilherme220044\Anaconda3\envs\Gene_rede\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Fitness_max' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\guilherme220044\Anaconda3\envs\Gene_rede\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individuo' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Hora de programar o algoritmo genético!



In [22]:
populacao = toolbox.populacao()

# É assim que calculamos a fitness dos individuos com DEAP
fitness = toolbox.map(toolbox.evaluate, populacao)

# Precisamos agora inserir essa informação nos nossos individuos
for ind, fit in zip(populacao, fitness):
    ind.fitness.values = fit

# Critério de parada neste caso é o número de gerações
for n in range(NUM_GERACOES):

    # Seleção
    proxima_geracao = toolbox.select(populacao, len(populacao))

    # Clone dos individuos (para evitar problemas com a forma que o python trabalha com listas)
    proxima_geracao = [toolbox.clone(ind) for ind in proxima_geracao]

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    for pai, mae in zip(pais, maes):
        if random.random() < CHANCE_CRUZAMENTO:
            toolbox.mate(pai, mae)

            # se cruzou, temos que deletar o fitness para calcular de novo
            del pai.fitness.values
            del mae.fitness.values

    # Mutação
    for possivel_mutante in proxima_geracao:
        if random.random() < CHANCE_MUTACAO:
            toolbox.mutate(possivel_mutante)

            # se mutou, temos que deletar o fitness para calcular de novo
            del possivel_mutante.fitness.values

    # Calcular o fitness de todos que mutaram ou cruzaram
    ind_sem_fitness = [ind for ind in proxima_geracao if not ind.fitness.valid]
    fitness = toolbox.map(toolbox.evaluate, ind_sem_fitness)
    for ind, fit in zip(ind_sem_fitness, fitness):
        ind.fitness.values = fit

    # Vamos atualizar a população!
    populacao[:] = proxima_geracao

    # Vamos atualizar o hall da fama
    hall_da_fama.update(populacao)

    # Vamos computar a estatística e atualizar o livro de registros
    estatistica_local = estatisticas.compile(populacao)
    log.record(gen=n + 1, nevals=len(ind_sem_fitness), **estatistica_local)
    # print(log.stream)
    if sum(hall_da_fama[0]) == NUM_CAIXAS:
        break

In [23]:
print("Melhor indivíduo já visto:")
print(hall_da_fama.items)

Melhor indivíduo já visto:
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
